 Preprocess and convert single-celldata in anndata (h5ad) format to loom for input to Geneformer
 - Uses same gene IDs that Geneformer was trained on
 - Need to provide .h5ad dataset with labels for finetuning
 - The preprocessing here is minimal and it is recommended to do other processing possibly including thresholding on reads, removing irrelevant cells types, or other steps specific to your application area
 - Change pathes/file names in Parameters section.


# Imports


In [1]:
import scanpy as sc
import os
import numpy as np
import pandas as pd
import glob
import anndata as ad
import pickle

# Download and Read input data to finetune the Geneformer pre-trained model

Many repositories have scRNAseq data in h5ad format, including:
 
   - [scPerturb](https://zenodo.org/record/7041849) Single-Cell Perturbation Database.
   
        example data downlaod from scPerturb: `!wget -P {input_data_filepath} https://zenodo.org/record/7041849/files/AissaBenevolenskaya2021.h5ad`
     
     
  -  [Cellxgene](https://cellxgene.cziscience.com/) Data Portal-  For example, the following is the paper and its corresponding data on Cellxgene. 
  
     - [Paper (Chen et al. 2021)](https://www.cell.com/cell/fulltext/S0092-8674(21)01381-7)
       
     - [Link to the data on Cellxgene](https://cellxgene.cziscience.com/collections/a48f5033-3438-4550-8574-cdff3263fdfd)
     
  **Note:** the URL is valid for one week. If it expires, get a fresh link from the Cellxgene data portal.
  
  -  Other publically available resources
  
          - Here, we use a small public data as a finetuning example. Other desired single-cell data in h5ad format can replace it.   

# Parameters

In [7]:
input_data_filepath = "/home/domino/geneformer_workflow/input/data/publicdata"
h5ad_filename = "adata_SS2_for_download.h5ad"#"GehringPachter2019.h5ad"#"AissaBenevolenskaya2021.h5ad"#"Chen_2021_VAL_DIS_Non_Epithelial.h5ad"

id_map_dir = "/home/domino/geneformer_workflow/input/id_map"

results_dir = "/home/domino/geneformer_workflow/results/perturbation"
os.makedirs(results_dir, exist_ok=True)

loom_dir = os.path.join(results_dir, "loom_files")
os.makedirs(loom_dir, exist_ok=True)
loom_filepath = os.path.join(loom_dir, "raw.loom")

adata_filepath = os.path.join(results_dir, 'adata.pk')

In [8]:
# Column name in adata for label to be used by geneformer finetuning
label_colname = "celltype"

In [9]:
adata = sc.read_h5ad(os.path.join(input_data_filepath, h5ad_filename))

In [10]:
adata

AnnData object with n_obs × n_vars = 1349 × 21030
    obs: 'batchcode', 'treatment', 'celltype', 'n_genes', 'lane', 'percent_mito', 'n_counts', 'scrublet_score', 'scrublet_cluster_score', 'bh_pval', 'batch'
    var: 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3'

In [11]:
adata.obs

,batchcode,treatment,celltype,n_genes,lane,percent_mito,n_counts,scrublet_score,scrublet_cluster_score,bh_pval,batch
SCGC--2500_C02,BATCH3,CONTROL,N,1264,24057_6,0.039233,18199.0,0.095477,0.095477,0.975326,0
SCGC--2499_C03,BATCH3,CONTROL,N,877,24057_6,0.080071,14000.0,0.028409,0.095477,0.975326,0
SCGC--2500_C03,BATCH3,CONTROL,N,1786,24057_6,0.054565,131017.0,0.257732,0.210526,0.808411,0
SCGC--2499_C04,BATCH3,CONTROL,N,394,24057_6,0.081609,14496.0,0.064000,0.083333,0.975326,0
SCGC--2500_C04,BATCH3,CONTROL,N,544,24057_6,0.099694,14364.0,0.109890,0.095477,0.975326,0
...,...,...,...,...,...,...,...,...,...,...,...
SCGC--2513_B10,BATCH1,CVID,SM,2177,24130_7,0.053583,385477.0,0.175573,0.234129,0.755951,3
SCGC--2510_B11,BATCH1,CONTROL,SM,1805,24130_7,0.056122,229909.0,0.210526,0.234129,0.755951,3
SCGC--2502_C01,BATCH1,CVID,N,303,24130_7,0.072749,1677.0,0.056180,0.056180,0.915471,3
SCGC--2502_C02,BATCH1,CVID,N,521,24130_7,0.085348,3351.0,0.032836,0.064000,0.915471,3


In [12]:
adata.var

,n_cells-0,n_cells-1,n_cells-2,n_cells-3
A1BG,53.0,57.0,36.0,50.0
A1BG-AS1,15.0,13.0,7.0,8.0
A1CF,5.0,8.0,NaN,9.0
A2M,9.0,12.0,7.0,8.0
A2ML1,5.0,6.0,3.0,10.0
...,...,...,...,...
ZYX,64.0,63.0,53.0,61.0
ZZEF1,77.0,94.0,72.0,79.0
ZZZ3,81.0,77.0,58.0,80.0
bP-21264C1.2,7.0,10.0,4.0,7.0


In [13]:
!wget -P {id_map_dir} https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/resolve/main/example_input_files/gene_info_table.csv

--2023-10-10 14:57:29--  https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/resolve/main/example_input_files/gene_info_table.csv
Resolving proxy-server.bms.com (proxy-server.bms.com)... 165.89.114.150
Connecting to proxy-server.bms.com (proxy-server.bms.com)|165.89.114.150|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 3460504 (3.3M) [text/plain]
Saving to: ‘/home/domino/geneformer_workflow/input/id_map/gene_info_table.csv’

gene_info_table.csv 100%[===================>]   3.30M  8.49MB/s    in 0.4s    

2023-10-10 14:57:29 (8.49 MB/s) - ‘/home/domino/geneformer_workflow/input/id_map/gene_info_table.csv’ saved [3460504/3460504]



In [14]:
gene_info = pd.read_csv(os.path.join(id_map_dir, "gene_info_table.csv"))
gene_info

,Unnamed: 0,ensembl_id,gene_name,gene_type
0,0,ENSG00000000003,TSPAN6,protein_coding
1,1,ENSG00000000005,TNMD,protein_coding
2,2,ENSG00000000419,DPM1,protein_coding
3,3,ENSG00000000457,SCYL3,protein_coding
4,4,ENSG00000000460,C1orf112,protein_coding
...,...,...,...,...
80524,80524,ENSGunknown_pk,pk,NaN
80525,80525,ENSGunknown_uc-338,uc-338,NaN
80526,80526,ENSGunknown_uc-338_1,uc-338.1,NaN
80527,80527,ENSGunknown_uc-338_2,uc-338.2,NaN


# Process data

In [15]:
# Remove gene names with multiple ensembl mappings
gene_info = gene_info.drop_duplicates(subset='gene_name', keep=False)

In [16]:
adata.var['gene_name'] = adata.var.index

In [17]:
adata.var

,n_cells-0,n_cells-1,n_cells-2,n_cells-3,gene_name
A1BG,53.0,57.0,36.0,50.0,A1BG
A1BG-AS1,15.0,13.0,7.0,8.0,A1BG-AS1
A1CF,5.0,8.0,NaN,9.0,A1CF
A2M,9.0,12.0,7.0,8.0,A2M
A2ML1,5.0,6.0,3.0,10.0,A2ML1
...,...,...,...,...,...
ZYX,64.0,63.0,53.0,61.0,ZYX
ZZEF1,77.0,94.0,72.0,79.0,ZZEF1
ZZZ3,81.0,77.0,58.0,80.0,ZZZ3
bP-21264C1.2,7.0,10.0,4.0,7.0,bP-21264C1.2


In [18]:
adata.var = adata.var.merge(gene_info[['gene_name', 'ensembl_id']],
                            on='gene_name',
                            how='left').set_index('gene_name')

In [19]:
adata.var

,n_cells-0,n_cells-1,n_cells-2,n_cells-3,ensembl_id
gene_name,,,,,
A1BG,53.0,57.0,36.0,50.0,ENSG00000121410
A1BG-AS1,15.0,13.0,7.0,8.0,ENSG00000268895
A1CF,5.0,8.0,NaN,9.0,ENSG00000148584
A2M,9.0,12.0,7.0,8.0,ENSG00000175899
A2ML1,5.0,6.0,3.0,10.0,ENSG00000166535
...,...,...,...,...,...
ZYX,64.0,63.0,53.0,61.0,ENSG00000159840
ZZEF1,77.0,94.0,72.0,79.0,ENSG00000074755
ZZZ3,81.0,77.0,58.0,80.0,ENSG00000036549


In [20]:
# Remove genes that didn't map to ensembl id
adata = adata[:, ~adata.var['ensembl_id'].isna()]

/opt/tbio/domino_202309/binaries/python-3.10.6/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [21]:
adata.var

,n_cells-0,n_cells-1,n_cells-2,n_cells-3,ensembl_id
gene_name,,,,,
A1BG,53.0,57.0,36.0,50.0,ENSG00000121410
A1BG-AS1,15.0,13.0,7.0,8.0,ENSG00000268895
A1CF,5.0,8.0,NaN,9.0,ENSG00000148584
A2M,9.0,12.0,7.0,8.0,ENSG00000175899
A2ML1,5.0,6.0,3.0,10.0,ENSG00000166535
...,...,...,...,...,...
ZYX,64.0,63.0,53.0,61.0,ENSG00000159840
ZZEF1,77.0,94.0,72.0,79.0,ENSG00000074755
ZZZ3,81.0,77.0,58.0,80.0,ENSG00000036549


In [22]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

/opt/tbio/domino_202309/binaries/python-3.10.6/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/opt/tbio/domino_202309/binaries/python-3.10.6/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/tbio/domino_202309/binaries/python-3.10.6/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [23]:
# Cells should be labeled with the total read count in the cell (column attribute "n_counts")
# to be used for normalization by geneformer.
adata.obs["n_counts"] = np.squeeze(np.asarray(np.sum(adata.X, axis=1))).astype("int")
adata.obs

,batchcode,treatment,celltype,n_genes,lane,percent_mito,n_counts,scrublet_score,scrublet_cluster_score,bh_pval,batch
SCGC--2500_C02,BATCH3,CONTROL,N,1161,24057_6,0.039233,16209,0.095477,0.095477,0.975326,0
SCGC--2499_C03,BATCH3,CONTROL,N,806,24057_6,0.080071,12543,0.028409,0.095477,0.975326,0
SCGC--2500_C03,BATCH3,CONTROL,N,1651,24057_6,0.054565,115491,0.257732,0.210526,0.808411,0
SCGC--2499_C04,BATCH3,CONTROL,N,347,24057_6,0.081609,12357,0.064000,0.083333,0.975326,0
SCGC--2500_C04,BATCH3,CONTROL,N,502,24057_6,0.099694,12333,0.109890,0.095477,0.975326,0
...,...,...,...,...,...,...,...,...,...,...,...
SCGC--2513_B10,BATCH1,CVID,SM,1983,24130_7,0.053583,330785,0.175573,0.234129,0.755951,3
SCGC--2510_B11,BATCH1,CONTROL,SM,1673,24130_7,0.056122,204929,0.210526,0.234129,0.755951,3
SCGC--2502_C01,BATCH1,CVID,N,289,24130_7,0.072749,1516,0.056180,0.056180,0.915471,3
SCGC--2502_C02,BATCH1,CVID,N,486,24130_7,0.085348,3097,0.032836,0.064000,0.915471,3


In [24]:
adata

AnnData object with n_obs × n_vars = 1333 × 19288
    obs: 'batchcode', 'treatment', 'celltype', 'n_genes', 'lane', 'percent_mito', 'n_counts', 'scrublet_score', 'scrublet_cluster_score', 'bh_pval', 'batch'
    var: 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3', 'ensembl_id', 'n_cells'

# Write output

In [25]:
# Write adata object
with open(adata_filepath, 'wb') as outfile:
    pickle.dump(adata, outfile, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
# Write out as loom as well
adata.write_loom(loom_filepath)

/home/domino/.local/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/home/domino/.local/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/home/domino/.local/lib/python3.10/site-pa